## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


#from os import path
#file_path = path.join("../Weather_Database/WeatherPy_database.csv")
#pd.read_csv(file_path).head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,43.00,71,100,3.00,moderate rain
1,1,Bluff,NZ,-46.6000,168.3333,55.99,78,100,1.99,overcast clouds
2,2,Kuusamo,FI,65.9667,29.1833,32.00,86,90,12.66,light snow
3,3,Tasiilaq,GL,65.6145,-37.6368,26.60,50,96,4.61,overcast clouds
4,4,Bambous Virieux,MU,-20.3428,57.7575,84.00,89,75,3.44,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bambous Virieux,MU,-20.3428,57.7575,84.00,89,75,3.44,broken clouds
8,8,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,3.44,broken clouds
9,9,Beira,MZ,-19.8436,34.8389,72.95,78,19,4.18,few clouds
10,10,Saint-Philippe,RE,-21.3585,55.7679,80.60,74,75,19.57,light rain
11,11,Poum,NC,-20.2333,164.0167,77.72,77,100,13.82,overcast clouds
12,12,Port Macquarie,AU,-31.4333,152.9167,73.40,64,73,13.80,broken clouds
13,13,Busselton,AU,-33.6500,115.3333,77.00,72,100,4.00,overcast clouds
18,18,Bowen,AU,-20.0167,148.2333,78.80,69,90,17.27,overcast clouds
19,19,Flinders,AU,-34.5833,150.8552,75.99,66,0,5.01,clear sky
23,23,Mityana,UG,0.4175,32.0228,71.60,78,20,13.80,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,84.00,broken clouds,-20.3428,57.7575,
8,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,
9,Beira,MZ,72.95,few clouds,-19.8436,34.8389,
10,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,
11,Poum,NC,77.72,overcast clouds,-20.2333,164.0167,
12,Port Macquarie,AU,73.40,broken clouds,-31.4333,152.9167,
13,Busselton,AU,77.00,overcast clouds,-33.6500,115.3333,
18,Bowen,AU,78.80,overcast clouds,-20.0167,148.2333,
19,Flinders,AU,75.99,clear sky,-34.5833,150.8552,
23,Mityana,UG,71.60,few clouds,0.4175,32.0228,


In [76]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={Lat},{Lng}&type={lodging}&key={g_key}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:  
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping")
    pass
   

Hotel not found... skipping


In [75]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,84.00,broken clouds,-20.3428,57.7575,Casa Tia Villa
8,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Beira,MZ,72.95,few clouds,-19.8436,34.8389,Beira Terrace Hotel
10,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Poum,NC,77.72,overcast clouds,-20.2333,164.0167,Gîte kajeon
...,...,...,...,...,...,...,...
677,Caninde,BR,70.90,overcast clouds,-4.3589,-39.3117,
681,Tehri,IN,80.60,clear sky,30.3833,78.4833,
683,Beloha,MG,76.98,clear sky,-25.1667,45.0500,
686,Algodones,MX,71.01,clear sky,20.3888,-105.4773,


In [97]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df != ''].dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,84.00,broken clouds,-20.3428,57.7575,Casa Tia Villa
8,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Beira,MZ,72.95,few clouds,-19.8436,34.8389,Beira Terrace Hotel
10,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Poum,NC,77.72,overcast clouds,-20.2333,164.0167,Gîte kajeon
12,Port Macquarie,AU,73.40,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
13,Busselton,AU,77.00,overcast clouds,-33.6500,115.3333,Observatory Guest House
18,Bowen,AU,78.80,overcast clouds,-20.0167,148.2333,Castle Motor Lodge Motel
19,Flinders,AU,75.99,clear sky,-34.5833,150.8552,Shellharbour Resort & Conference Centre
23,Mityana,UG,71.60,few clouds,0.4175,32.0228,Green Valley Hotel


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
